### QAOA

Quantum Approximate Optimization Algorithm

In [3]:
import numpy as np

def check_hermitian(h):
    adjoint = h.conj().T # a.k.a. conjugate-transpose, transjugate, dagger
    return np.array_equal(h,adjoint)

def matrixify(n_qubits,wsopp):
    """
        wsopp: Weighted Sum-of-Product of Paulis
    """
    I = np.array([[1,0],[0,1]])
    X = np.array([[0,1],[1,0]])
    Y = np.array([[0,complex(0,-1)],[complex(0,1),0]])
    Z = np.array([[1,0],[0,-1]])
    hamiltonian = np.zeros([2**n_qubits,2**n_qubits])
    for wpp in wsopp:
        ptm = [1]
        for pt in wpp[1]:
            if pt == "X":
                ptm = np.kron(ptm,X)
            elif pt == "Y":
                ptm = np.kron(ptm,Y)
            elif pt == "Z":
                ptm = np.kron(ptm,Z)
            else: # Identity
                ptm = np.kron(ptm,I)
        hamiltonian += wpp[0]*ptm
    assert(check_hermitian(hamiltonian))
    return hamiltonian

In [1]:
from scipy.optimize import minimize
import re
from qxelarator import qxelarator
from functools import reduce
import numpy as np

class QAOA(object):

    def get_angles(self, n_qubits, ansatz, wsopp, betas, gammas, ang_id, coeffs, steps):
        v = VQE()
        return v.vqeqaoa_run(wsopp, ansatz, n_qubits, steps, np.hstack((betas, gammas)), ang_id, coeffs)       

    def probabilities(ang):
        # Computes the probability of each state given a particular set of angles.
        prog = "test_output/qaoa_try.qasm"
        probs = []
        # RUN AND MEASURE ALL n QUBITS, TO DETERINE PROBABILITY OF ALL 2^n STATES
        return probs

In [4]:
import numpy as np
import networkx as nx

def graph_to_pqasm(g):
    # Specific for Max-Cut Hamiltonian
    # PauliTerm to Gates concept from rigetti/pyquil/pyquil/paulis.py
    coeffs = [] # Weights for the angle parameter for each gate
    angles = [0,0]
    wsopp = [] # {weight | pauliProduct}
    n_qubit = 3
    Iall = "III"
    ansatz = [] # qasm tokens
    for i,j in g.edges():
        # 0.5*Z_i*Z_j
        ansatz.append(("cnot",[i,j]))
        ansatz.append(("rz",i))
        coeffs.append(2*0.5)
        angles[0] += 1 # beta
        ansatz.append(("cnot",[i,j]))
        sopp = Iall[:n_qubit-1-i]+"Z"+Iall[n_qubit-1-i+1:]
        sopp = sopp[:n_qubit-1-j]+"Z"+sopp[n_qubit-1-j+1:]
        wsopp.append((0.5,sopp))
        wsopp.append((-0.5,Iall))
        # -0.5*I_0
        ansatz.append(("x",0))
        ansatz.append(("rz",0))
        coeffs.append(-1*0.5)
        angles[0] += 1 # beta
        ansatz.append(("x",0))
        ansatz.append(("rz",0))
        coeffs.append(-1*0.5)
        angles[0] += 1 # beta
    for i in g.nodes():
        # -X_i
        ansatz.append(("h",i))
        ansatz.append(("rz",i))
        coeffs.append(2*-1)
        angles[1] += 1 # gamma
        ansatz.append(("h",i))
    return wsopp, ansatz, coeffs, angles
    
###################################################################################################################

# Barbell graph

g = nx.Graph()
g.add_edge(0,1)
g.add_edge(1,2)

n_qubits = len(g.nodes())

wsopp, stencil, cfs, aid = graph_to_pqasm(g)

steps = 2
ansatz = []
coeffs = []
# Reference state preparation
for i in range(0,n_qubits):
    ansatz.append(("h",i))
# Repeat graph ansatz for specified steps
for i in range(0,steps):
    for gate in stencil:
        ansatz.append(gate)
    coeffs = np.hstack((coeffs,cfs))
init_betas = np.random.uniform(0, np.pi, steps) # Initial beta angle parameters of cost Hamiltonian
init_gammas = np.random.uniform(0, 2*np.pi, steps) # Initial gamma angle parameters of driving/mixing Hamiltonian
    
###################################################################################################################

hamiltonian = matrixify(n_qubits, wsopp)
print("wsopp = ",wsopp,"\nhamiltonian = ",hamiltonian,"\nAnsatz: ")
for i in ansatz:
    print(i)
print("Init_Betas = ",init_betas,"\nInit_Gammas = ",init_gammas,"\nAngle ID = ",aid,"\nCoefficients = ",coeffs,"\nSteps = ",steps)
      
###################################################################################################################

qaoa_obj = QAOA()
#r = qaoa_obj.get_angles(n_qubits,wsopp,ansatz,init_betas,init_gammas,aid,coeffs,steps)
#print(r.status, r.fun, r.x)

# The last qaoa_try will have the optimal angles
#probs = qaoa_obj.probabilities()
#print(probs)

wsopp =  [(0.5, 'IZZ'), (-0.5, 'III'), (0.5, 'ZZI'), (-0.5, 'III')] 
hamiltonian =  [[ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -2.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -2.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]] 
Ansatz: 
('h', 0)
('h', 1)
('h', 2)
('cnot', [0, 1])
('rz', 0)
('cnot', [0, 1])
('x', 0)
('rz', 0)
('x', 0)
('rz', 0)
('cnot', [1, 2])
('rz', 1)
('cnot', [1, 2])
('x', 0)
('rz', 0)
('x', 0)
('rz', 0)
('h', 0)
('rz', 0)
('h', 0)
('h', 1)
('rz', 1)
('h', 1)
('h', 2)
('rz', 2)
('h', 2)
('cnot', [0, 1])
('rz', 0)
('cnot', [0, 1])
('x', 0)
('rz', 0)
('x', 0)
('rz', 0)
('cnot', [1, 2])
('rz', 1)
('cnot', [1, 2])
('x', 0)
('rz', 0)
('x', 0)
('rz', 0)
('h', 0)
('rz', 0)
('h', 0)
('h', 1)
('rz', 1)
('h', 1)
('h', 2)
('rz', 2)
('h', 2)
Init_Betas =  [1.84974234 0.07048545] 
Init_Gammas =  [3.21163919 2.33448204] 
Angle ID =  [6, 3] 
Coe